<a href="https://colab.research.google.com/github/Diamantinomc/python-scripts/blob/master/consumos_epp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear un script que cuente la cantidd de consumos en los que se piden Epp's
- Leer archivo
- Agregar fecha entre la que buscar
- Encontrar consumos con epp's (Si se hace desde el archivo de ropa este paso se salta, ya que todos son de epp)
- Contar cuantos consumos hay
- Contar cuantas veces se repite cada artículo
- Separar por centro de costo
- Generar gráficos
- Entregar información en un pdf

In [ ]:
!pip install fpdf2 plotly kaleido seaborn pandas matplotlib

# Importaciones
import pandas as pd
from datetime import datetime
from google.colab import files
import matplotlib.pyplot as plt
import os
import plotly.express as px
from fpdf import FPDF
import seaborn as sns
import numpy as np
import sys

# Verificamos que kaleido esté instalado
try:
    import kaleido
    print("✅ Kaleido está correctamente instalado")
except ImportError:
    print("❌ Kaleido no está instalado, intentando reinstalar...")
    !pip install -q kaleido
    import kaleido

# ================== CONFIGURACIÓN ==================
AREAS = {
    '2162':'Mantención General',
    '2172':'Mantención Líneas',
    '2112':'Bodega',
    '2106':'Postcosecha',
    '2012':'Abastecimiento',
    '2114':'Materia Prima',
    '2115':'Producto Terminado',
    '2192':'Packing',
    '2111':'Sadema',
    '2508': 'Gerencia Industrial',
    '2512':'SAG'
}

COLUMNAS_AGRUPACION = [
    'CÓDIGO ARTÍCULO',
    'NOMBRE DEL PRODUCTO',
    'CONFIGURACION',
    'TALLA'
]

# Configuración de estilos
plt.style.use('seaborn-v0_8')
sns.set_theme(style="whitegrid")
COLORES = px.colors.qualitative.Plotly

# ================== FUNCIONES AUXILIARES ==================
def validar_fecha(texto_fecha):
    """Valida el formato de fecha dd-mm-yyyy y retorna datetime"""
    try:
        return datetime.strptime(texto_fecha, '%d-%m-%Y')
    except ValueError:
        print("¡Error! Usa el formato dd-mm-yyyy (ej: 31-12-2023).")
        return None

def cargar_datos():
    """Carga y retorna el DataFrame con los datos"""
    uploaded = files.upload()
    archivo = list(uploaded.keys())[0]
    return pd.read_excel(archivo, sheet_name="Datos Planta", engine='openpyxl')

def filtrar_por_fechas(df, fecha_inicio, fecha_fin):
    """Filtra el DataFrame por rango de fechas"""
    df['FECHA DE DOCUMENTO'] = pd.to_datetime(df['FECHA DE DOCUMENTO'], format='%d-%m-%Y')
    filtro = (df['FECHA DE DOCUMENTO'] >= fecha_inicio) & (df['FECHA DE DOCUMENTO'] <= fecha_fin)
    return df.loc[filtro]

# ================== PROCESAMIENTO DE DATOS ==================
def contar_documentos_unicos(df):
    """Cuenta documentos únicos en el DataFrame filtrado"""
    return df['N° DE DOCUMENTO'].nunique()

def procesar_articulos(df):
    """Agrupa y suma artículos, ordena por cantidad"""
    conteo = df.groupby(['CÓDIGO ARTÍCULO', 'NOMBRE DEL PRODUCTO'])['CANTIDAD DE SALIDA'] \
              .sum().reset_index(name='Entregas')
    conteo['Entregas'] = conteo['Entregas'].astype(int)
    return conteo.sort_values('Entregas', ascending=False)

def procesar_por_area(df):
    """Agrupa por centro de costo y artículo, mapea áreas"""
    conteo = df.groupby(['CENTRO DE COSTO', 'CÓDIGO ARTÍCULO', 'NOMBRE DEL PRODUCTO', 'N° DE DOCUMENTO']) \
              ['CANTIDAD DE SALIDA'].sum().reset_index(name='Cantidad total')
    conteo['Cantidad total'] = conteo['Cantidad total'].astype(int)
    conteo['NOMBRE AREA'] = conteo['CENTRO DE COSTO'].astype(str).map(AREAS)
    return conteo.sort_values(['CENTRO DE COSTO', 'Cantidad total'], ascending=[True, False])

def procesar_por_caracteristicas(df):
    """Agrupa por características específicas del artículo"""
    conteo = df.groupby(COLUMNAS_AGRUPACION)['CANTIDAD DE SALIDA'] \
              .sum().reset_index(name='Cantidad total')
    conteo['Cantidad total'] = conteo['Cantidad total'].astype(int)
    return conteo.sort_values(['CÓDIGO ARTÍCULO', 'Cantidad total'], ascending=[True, False])

# ================== GENERACIÓN DE GRÁFICOS ==================
def generar_graficos_avanzados(resultados):
    """Genera gráficos profesionales con Seaborn y Plotly"""
    os.makedirs('graficos', exist_ok=True)

    # 1. Gráfico de documentos por área (Seaborn)
    plt.figure(figsize=(14, 7))
    datos_areas = resultados['por_area'].groupby('NOMBRE AREA')['N° DE DOCUMENTO'].nunique().sort_values(ascending=False)
    ax = sns.barplot(
        x=datos_areas.index,
        y=datos_areas.values,
        palette="Blues_d"
    )
    plt.title('Documentos Únicos por Área', fontsize=16, pad=20)
    plt.xlabel('Área', fontsize=14)
    plt.ylabel('Cantidad', fontsize=14)
    plt.xticks(rotation=45, ha='right')

    for p in ax.patches:
        ax.annotate(f'{int(p.get_height())}',
                   (p.get_x() + p.get_width() / 2., p.get_height()),
                   ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                   textcoords='offset points')

    plt.tight_layout()
    plt.savefig('graficos/documentos_por_area.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Gráfico circular mejorado (Plotly)
    try:
        top_10 = resultados['articulos'].head(10)
        fig = px.pie(top_10,
                     values='Entregas',
                     names='NOMBRE DEL PRODUCTO',
                     title='Top 10 Artículos Más Entregados',
                     hole=0.3,
                     color_discrete_sequence=px.colors.sequential.Blues_r)

        fig.update_traces(textposition='inside', textinfo='percent+label')

        # Verificamos explícitamente kaleido
        import kaleido
        fig.write_image('graficos/top_articulos.png', scale=2, engine='kaleido')
    except Exception as e:
        print(f"⚠️ Error con Plotly/Kaleido: {str(e)}")
        print("🔹 Generando versión alternativa con matplotlib...")
        plt.figure(figsize=(10, 8))
        plt.pie(top_10['Entregas'],
                labels=top_10['NOMBRE DEL PRODUCTO'],
                autopct='%1.1f%%',
                startangle=90)
        plt.title('Top 10 Artículos Más Entregados', pad=20)
        plt.savefig('graficos/top_articulos.png', dpi=300, bbox_inches='tight')
        plt.close()

    # 3. Gráficos combinados interactivos (Plotly)
    for codigo in resultados['por_caracteristicas']['CÓDIGO ARTÍCULO'].unique()[:5]:
        df_art = resultados['por_caracteristicas'][resultados['por_caracteristicas']['CÓDIGO ARTÍCULO'] == codigo]
        nombre = df_art['NOMBRE DEL PRODUCTO'].iloc[0]

        try:
            fig = px.bar(df_art,
                         x='CONFIGURACION',
                         y='Cantidad total',
                         color='TALLA',
                         barmode='group',
                         title=f'{nombre} - Distribución por Configuración y Talla',
                         labels={'CONFIGURACION': 'Configuración', 'Cantidad total': 'Total Entregado'},
                         color_discrete_sequence=COLORES)

            fig.update_layout(
                plot_bgcolor='white',
                hovermode='x unified',
                legend=dict(orientation="h", yanchor="bottom", y=1.02),
                font=dict(size=12)
            )

            fig.write_html(f'graficos/interactivo_{codigo}.html')
        except Exception as e:
            print(f"⚠️ Error al generar gráfico interactivo para {nombre}: {str(e)}")

# ================== GENERACIÓN DE PDF ==================
def generar_informe_pdf(resultados, filename="informe_entregas.pdf"):
    """Genera un PDF profesional con gráficos y estadísticas"""
    class PDF(FPDF):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # Descargar e instalar una fuente Unicode compatible
            self.add_font('DejaVu', '', 'DejaVuSans.ttf', uni=True)
            self.add_font('DejaVu', 'B', 'DejaVuSans-Bold.ttf', uni=True)
            self.add_font('DejaVu', 'I', 'DejaVuSans-Oblique.ttf', uni=True)

        def header(self):
            self.set_font('DejaVu', 'B', 16)
            self.cell(0, 10, 'Informe de Entregas de Artículos', 0, 1, 'C')
            self.ln(10)

        def footer(self):
            self.set_y(-15)
            self.set_font('DejaVu', 'I', 8)
            self.cell(0, 10, f'Página {self.page_no()}', 0, 0, 'C')

    # Descargar las fuentes DejaVu (compatibles con Unicode)
    import os
    if not os.path.exists('DejaVuSans.ttf'):
        !wget https://github.com/dejavu-fonts/dejavu-fonts/releases/download/version_2_37/dejavu-fonts-ttf-2.37.zip
        !unzip dejavu-fonts-ttf-2.37.zip
        !mv dejavu-fonts-ttf-2.37/ttf/DejaVuSans.ttf .
        !mv dejavu-fonts-ttf-2.37/ttf/DejaVuSans-Bold.ttf .
        !mv dejavu-fonts-ttf-2.37/ttf/DejaVuSans-Oblique.ttf .

    pdf = PDF()
    pdf.add_page()

    # Configurar la fuente Unicode para todo el documento
    pdf.set_font('DejaVu', '', 12)

    # 1. Portada
    pdf.set_font('DejaVu', 'B', 18)
    pdf.cell(0, 15, 'INFORME DE ENTREGAS', 0, 1, 'C')
    pdf.ln(10)

    pdf.set_font('DejaVu', '', 14)
    pdf.cell(0, 10, f"Período: {resultados['fecha_inicio']} a {resultados['fecha_fin']}", 0, 1)
    pdf.cell(0, 10, f"Documentos únicos: {resultados['documentos_unicos']}", 0, 1)
    pdf.ln(15)

    # 2. Todos los Artículos (con paginación automática)
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 10, 'Listado Completo de Artículos Entregados', 0, 1)
    pdf.ln(8)

    # Tabla de artículos con encabezado
    pdf.set_font('DejaVu', 'B', 12)
    pdf.set_fill_color(200, 220, 255)
    pdf.cell(40, 10, 'Código', 1, 0, 'C', 1)
    pdf.cell(100, 10, 'Artículo', 1, 0, 'C', 1)
    pdf.cell(30, 10, 'Cantidad', 1, 1, 'C', 1)
    pdf.set_font('DejaVu', '', 11)

    for _, row in resultados['articulos'].iterrows():
        # Verificar si necesitamos nueva página
        if pdf.get_y() > 250:
            pdf.add_page()
            # Repetir encabezado de tabla en nueva página
            pdf.set_font('DejaVu', 'B', 12)
            pdf.set_fill_color(200, 220, 255)
            pdf.cell(40, 10, 'Código', 1, 0, 'C', 1)
            pdf.cell(100, 10, 'Artículo', 1, 0, 'C', 1)
            pdf.cell(30, 10, 'Cantidad', 1, 1, 'C', 1)
            pdf.set_font('DejaVu', '', 11)

        pdf.cell(40, 8, str(row['CÓDIGO ARTÍCULO']), 1, 0)
        pdf.cell(100, 8, row['NOMBRE DEL PRODUCTO'], 1, 0)
        pdf.cell(30, 8, str(row['Entregas']), 1, 1, 'C')

    # 3. Gráficos estáticos
    pdf.add_page()
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 10, 'Distribución por Áreas', 0, 1)
    pdf.image('graficos/documentos_por_area.png', x=10, y=25, w=180)

    pdf.add_page()
    pdf.cell(0, 10, 'Top Artículos', 0, 1)
    pdf.image('graficos/top_articulos.png', x=30, y=25, w=140)

    # 4. Estadísticas
    pdf.add_page()
    pdf.set_font('DejaVu', 'B', 16)
    pdf.cell(0, 10, 'Estadísticas Clave', 0, 1)
    pdf.ln(10)

    stats = [
        ("• Total artículos diferentes", resultados['articulos']['CÓDIGO ARTÍCULO'].nunique()),
        ("• Área con más pedidos", resultados['por_area'].groupby('NOMBRE AREA')['Cantidad total'].sum().idxmax()),
        ("• Artículo más solicitado", resultados['articulos'].iloc[0]['NOMBRE DEL PRODUCTO']),
        ("• Cantidad total entregada", resultados['articulos']['Entregas'].sum())
    ]

    pdf.set_font('DejaVu', '', 12)
    for stat, value in stats:
        pdf.cell(0, 10, f"{stat}: {value}", 0, 1)

    pdf.output(filename)
    return filename

# ================== FUNCIÓN PRINCIPAL ==================
def main():
    """Función principal que orquesta todo el proceso"""
    try:
        # 1. Carga de datos
        print("📊 INICIANDO PROCESAMIENTO DE DATOS...")
        df = cargar_datos()

        # 2. Obtener fechas
        print("\n📅 INGRESE RANGO DE FECHAS (DD-MM-YYYY):")
        while True:
            fecha_inicio = validar_fecha(input("Fecha de inicio: "))
            if fecha_inicio is not None: break

        while True:
            fecha_fin = validar_fecha(input("Fecha de término: "))
            if fecha_fin is not None: break

        # 3. Filtrado
        df_filtrado = filtrar_por_fechas(df, fecha_inicio, fecha_fin)

        # 4. Procesamiento
        resultados = {
            'fecha_inicio': fecha_inicio.strftime('%d-%m-%Y'),
            'fecha_fin': fecha_fin.strftime('%d-%m-%Y'),
            'documentos_unicos': contar_documentos_unicos(df_filtrado),
            'articulos': procesar_articulos(df_filtrado),
            'por_area': procesar_por_area(df_filtrado),
            'por_caracteristicas': procesar_por_caracteristicas(df_filtrado)
        }

        return resultados

    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        return None

# ================== EJECUCIÓN ==================
if __name__ == "__main__":
    # Ejecutar procesamiento principal
    datos = main()

    if datos:
        # Generar visualizaciones
        print("\n🎨 GENERANDO GRÁFICOS PROFESIONALES...")
        generar_graficos_avanzados(datos)

        # Generar PDF
        print("\n📄 CREANDO INFORME PDF...")
        archivo_pdf = generar_informe_pdf(datos)

        # Resultados finales
        print("\n✅ PROCESO COMPLETADO CON ÉXITO")
        print(f"📂 Gráficos guardados en: {os.getcwd()}/graficos/")
        print(f"📄 Informe PDF generado: {archivo_pdf}")

        # Descargar resultados
        files.download(archivo_pdf)
        for file in os.listdir('graficos'):
            if file.endswith('.html'):
                files.download(f'graficos/{file}')

